# Cashflow Model

This notebook imports a csv listing of uuids with potential affordability issues and assesses affordability using their historical open banking data.   Customer outgoings are increased for inflation. Customers going into overdraft too frequently are added to a 'failed' list.  The dataframes are exported to an excel file for review.

In [1]:
import numpy as np 
import pandas as pd

In [2]:
# This cell will import the csv file listing customer references needing additional affordability checks:

affordcheck_df = pd.read_excel('referred_applicants_report.xlsx')

#import os
#current_dir = os.getcwd()
#parent_dir = os.path.dirname(current_dir)
#subfolder = "4 approval_models"
#file1 = "referred_applicants_2023-09-02_16-30-13.xlsx"  

#file_path1 = os.path.join(parent_dir, subfolder,file1)
#affordcheck_df = pd.read_excel(file_path1)

affordcheck_df.head()

,uuid,amount_requested,duration_requested,interest_rate
0,d8c5c75e-a5d0-462c-b6aa-634877364f74,10000,60,24.6
1,bf5637ac-afcd-4418-b8b7-7dc9a13c0bf0,9500,48,24.6
2,cb7b6ec0-a013-445e-a0dc-68114eca1251,10000,18,24.6
3,f838ad04-03cc-4543-ac16-4d39d78f8b4c,10000,48,24.6
4,bf2b9cb0-927a-4313-945f-72cadabe0e5c,6000,24,24.6


In [3]:
# This cell calculates the monthly payments for customers requested loan amount and estimated interest rate
# Monthly amount is added onto the data file just imported

#!pip install numpy_financial
import numpy_financial as npf

interest_rate = affordcheck_df['interest_rate']
monthly_rate = np.power(1 + interest_rate, 1/12) - 1
amount = affordcheck_df['amount_requested']
period = affordcheck_df['duration_requested']

def PMT(uuid):
    index = affordcheck_df[affordcheck_df['uuid'] == uuid].index[0]  # Find the index for the specific UUID
    pmt = npf.pmt(monthly_rate.iloc[index], period.iloc[index], -amount.iloc[index])
    return pmt.round(0)

affordcheck_df['PMT'] = affordcheck_df['uuid'].apply(PMT)
affordcheck_df.head(2)

,uuid,amount_requested,duration_requested,interest_rate,PMT
0,d8c5c75e-a5d0-462c-b6aa-634877364f74,10000,60,24.6,3102.0
1,bf5637ac-afcd-4418-b8b7-7dc9a13c0bf0,9500,48,24.6,2947.0


In [4]:
# This cell imports the full set of open banking data:
open_banking_df = pd.read_csv('summary_block.csv')

#subfolder = "1 raw_data"
#file2 = "summary_block.csv"
#file_path2 = os.path.join(parent_dir, subfolder,file2)
#open_banking_df = pd.read_csv(file_path2)

In [5]:
# Check duplicates
duplicates = open_banking_df.duplicated()
duplicates.sum()

0

In [6]:
# This cell drops columns in the Open Banking data that are not required
columns_to_retain = ['uuid', 'date_created', 'avg_balance_last_360_days',
    'total_income_broad_last_30_days', 'total_income_broad_last_90_days', 'total_income_broad_last_180_days', 'total_income_broad_last_360_days',
    'total_benefits_last_30_days', 'total_benefits_last_90_days', 'total_benefits_last_180_days', 'total_benefits_last_360_days',
    'net_groceries_spend_last_30_days', 'net_groceries_spend_last_90_days', 'net_groceries_spend_last_180_days', 'net_groceries_spend_last_360_days',
    'net_mortgage_and_rent_spend_last_30_days', 'net_mortgage_and_rent_spend_last_90_days', 'net_mortgage_and_rent_spend_last_180_days','net_mortgage_and_rent_spend_last_360_days',
    'net_bills_and_utilities_spend_last_30_days', 'net_bills_and_utilities_spend_last_90_days', 'net_bills_and_utilities_spend_last_180_days','net_bills_and_utilities_spend_last_360_days',
    'total_gambling_spend_last_30_days','total_gambling_spend_last_90_days', 'total_gambling_spend_last_180_days', 'total_gambling_spend_last_360_days',
    'total_payday_spend_last_30_days', 'total_payday_spend_last_90_days', 'total_payday_spend_last_180_days', 'total_payday_spend_last_360_days',
    #'net_savings_spend_last_30_days','net_savings_spend_last_90_days','net_savings_spend_last_180_days', 'net_savings_spend_last_360_days'
]

# Select the desired columns and drop the rest
open_banking_df2 = open_banking_df.loc[:, columns_to_retain]
open_banking_df2.head(1)

,uuid,date_created,avg_balance_last_360_days,total_income_broad_last_30_days,total_income_broad_last_90_days,total_income_broad_last_180_days,total_income_broad_last_360_days,total_benefits_last_30_days,total_benefits_last_90_days,total_benefits_last_180_days,...,net_bills_and_utilities_spend_last_180_days,net_bills_and_utilities_spend_last_360_days,total_gambling_spend_last_30_days,total_gambling_spend_last_90_days,total_gambling_spend_last_180_days,total_gambling_spend_last_360_days,total_payday_spend_last_30_days,total_payday_spend_last_90_days,total_payday_spend_last_180_days,total_payday_spend_last_360_days
0,7c595a50-af02-49d9-9d41-06ec90fd6a6a,10/07/2021,-1059.0,1414.49,4175.5,7828.54,16010.79,0.0,0.0,0.0,...,-889.0,-1710.22,0.0,-80.0,-150.0,-300.0,0.0,0.0,-891.18,-2717.46


In [7]:
# This cell filters the open banking data so that only the rows for customer uuids contained in File 1 are retained 
# A list of any customers missing Open Banking data is generated

unique_references = affordcheck_df['uuid'].tolist()
subset_open_banking_df = open_banking_df2[open_banking_df2['uuid'].isin(unique_references)]
#print(subset_open_banking_df.head(1))

missing_customers = affordcheck_df[~affordcheck_df['uuid'].isin(subset_open_banking_df['uuid'])]
if not missing_customers.empty:
    print("Missing customers in Open Banking data:")
    print(missing_customers)
else:
    print("All customers are present in Open Banking data.")

# Create a new list of unique UUIDs excluding the missing customers
valid_references = [uuid for uuid in unique_references if uuid not in missing_customers['uuid'].tolist()]
print("Valid customers based on Open Banking data:")
print(valid_references[:5])

# If you want to view the full list of uuid's print the line below
#print(valid_references)

Missing customers in Open Banking data:
                                      uuid  amount_requested  \
0     d8c5c75e-a5d0-462c-b6aa-634877364f74             10000   
3     f838ad04-03cc-4543-ac16-4d39d78f8b4c             10000   
5     f71acafb-8c70-4b06-ac85-be97335cc254             15000   
6     832d4269-d933-4c5a-81ef-cea8daad2c21              1500   
8     4737d78f-f4bf-4eeb-93ed-f66f96122424              2000   
...                                    ...               ...   
4568  590f9e77-42c8-4117-9d36-b3cde8040c2b             13000   
4569  fb3e2061-25ce-4f0a-ad4c-023a36ea10c0             25000   
4570  eadb668f-f610-4b19-aa38-e15eb22b5418             12000   
4572  011bfd2e-b433-4583-b40d-2fb58f1050ef             15000   
4575  594e1623-b3f0-44d5-ac8d-bf11e3f01d83             10000   

      duration_requested  interest_rate     PMT  
0                     60           24.6  3102.0  
3                     48           24.6  3102.0  
5                     36           24.6  

In [8]:
# This cell imports the external inflation data 
inflation_df = pd.read_excel('boe_inflation_forecast.xlsx', index_col=0)

#subfolder3 = "5 cashflow_models"
#file3 = "boe_inflation_forecast.xlsx"
#sheet_name = 'Export' 
#file_path3 = os.path.join(parent_dir, subfolder3, file3)

#inflation_df = pd.read_excel(file_path3,sheet_name=sheet_name,index_col=0)
inflation_df.head()

,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,Month 9,Month 10,...,Month 15,Month 16,Month 17,Month 18,Month 19,Month 20,Month 21,Month 22,Month 23,Month 24
Measure,,,,,,,,,,,,,,,,,,,,,
CPI,0.005576,0.003994,0.003994,0.003994,0.003515,0.003515,0.003515,0.002709,0.002709,0.002709,...,0.002060,0.002060,0.001815,0.001815,0.001815,0.001406,0.001406,0.001406,0.001652,0.001652
CPIH,0.005855,0.004194,0.004194,0.004194,0.003690,0.003690,0.003690,0.002845,0.002845,0.002845,...,0.002163,0.002163,0.001906,0.001906,0.001906,0.001476,0.001476,0.001476,0.001734,0.001734
Wages,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000
Benefits,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000


In [9]:
print(inflation_df.index)

Index(['CPI', 'CPIH', 'Wages', 'Benefits'], dtype='object', name='Measure')


In [10]:
# This cell creates an empty template for the cashflow analysis to be performed on each customer
# Create a dataFrame template for the cashflow

columns = [
    'Month -6', 'Month -5', 'Month -4', 'Month -3', 'Month -2', 'Month -1', 'Month 1', 'Month 2', 'Month 3', 'Month 4', 'Month 5','Month 6', 'Month 7', 'Month 8', 'Month 9', 'Month 10', 'Month 11', 'Month 12',
    'Month 13', 'Month 14', 'Month 15', 'Month 16', 'Month 17','Month 18', 'Month 19', 'Month 20', 'Month 21', 'Month 22', 'Month 23','Month 24'
]

index = [
    'uuid',
    'B/f',
    'Broad income', 
    'Total benefits',
    'Total income',
    'Groceries spend',
    'Mortgage & rent',
    'Bills & utilities',
    'Gambling spend',
    'Payday spend',
    'Total spending',
    'EDI',
    'LOAN PMT',
    'C/f',
]
              
data = np.zeros((len(index), len(columns)))

cashflow_template = pd.DataFrame(data, index=index, columns=columns)


In [11]:
# This cell iterates through the valid, unique UUIDs listed on the affordcheck_df for which Open Banking data is available 
# and calculates the cash flows for each
# Transfers to savings accounts have been excluded from the cashflow

dataframe_outputs = []  # A list of dataframes will be created by the loop
failed_affordability = []
gamblers = []
drop_in_essential_spending = []

for target_uuid in valid_references:
    selected_row = subset_open_banking_df[subset_open_banking_df['uuid'] == target_uuid]
    
    # These rows add the uuid for each iteration
    cashflow = cashflow_template.copy()
    cashflow.rename(index={'uuid': target_uuid}, inplace=True)
        
    # These rows specify the figures to insert into the cashflow for each uuid
    avg_balance = selected_row['avg_balance_last_360_days'].values[0]
    cashflow.loc['B/f', 'Month -6'] = avg_balance 
    
    historical_periods = [-6, -5, -4, -3, -2, -1]
    fcast_periods = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,13,14,15,16,17,18,19,20,21,22,23,24]

    for i in historical_periods:
        if i == -1:
            cashflow.loc['Broad income', f'Month {i}'] = selected_row['total_income_broad_last_30_days'].iloc[0]
            cashflow.loc['Total benefits', f'Month {i}'] = selected_row['total_benefits_last_30_days'].iloc[0]
            cashflow.loc['Groceries spend',f'Month {i}'] = selected_row['net_groceries_spend_last_30_days'].iloc[0]
            cashflow.loc['Mortgage & rent',f'Month {i}'] = selected_row['net_mortgage_and_rent_spend_last_30_days'].iloc[0]
            cashflow.loc['Bills & utilities',f'Month {i}'] = selected_row['net_bills_and_utilities_spend_last_30_days'].iloc[0]
            cashflow.loc['Gambling spend',f'Month {i}'] = selected_row['total_gambling_spend_last_30_days'].iloc[0]
            cashflow.loc['Payday spend',f'Month {i}'] = selected_row['total_payday_spend_last_30_days'].iloc[0]     
        elif i == -3:
            cashflow.loc['Broad income', f'Month {i}'] = (selected_row['total_income_broad_last_90_days'].iloc[0] - selected_row['total_income_broad_last_30_days'].iloc[0]) / 2
            cashflow.loc['Total benefits', f'Month {i}'] = (selected_row['total_benefits_last_90_days'].iloc[0] - selected_row['total_benefits_last_30_days'].iloc[0])/2
            cashflow.loc['Groceries spend', f'Month {i}'] = (selected_row['net_groceries_spend_last_90_days'].iloc[0] - selected_row['net_groceries_spend_last_30_days'].iloc[0])/2 
            cashflow.loc['Mortgage & rent', f'Month {i}'] = (selected_row['net_mortgage_and_rent_spend_last_90_days'].iloc[0] - selected_row['net_mortgage_and_rent_spend_last_30_days'].iloc[0])/2
            cashflow.loc['Bills & utilities',f'Month {i}'] = (selected_row['net_bills_and_utilities_spend_last_90_days'].iloc[0] - selected_row['net_bills_and_utilities_spend_last_30_days'].iloc[0])/2
            cashflow.loc['Gambling spend',f'Month {i}'] = (selected_row['total_gambling_spend_last_90_days'].iloc[0] - selected_row['total_gambling_spend_last_30_days'].iloc[0])/2
            cashflow.loc['Payday spend',f'Month {i}'] = (selected_row['total_payday_spend_last_90_days'].iloc[0] - selected_row['total_payday_spend_last_30_days'].iloc[0])/2 
        elif i == -2:
            cashflow.loc['Broad income', f'Month {i}'] = cashflow.loc['Broad income', f'Month -3']
            cashflow.loc['Total benefits', f'Month {i}'] = cashflow.loc['Total benefits', f'Month -3']
            cashflow.loc['Groceries spend', f'Month {i}'] = cashflow.loc['Groceries spend', f'Month -3']
            cashflow.loc['Mortgage & rent', f'Month {i}'] = cashflow.loc['Mortgage & rent', f'Month -3']
            cashflow.loc['Bills & utilities', f'Month {i}'] = cashflow.loc['Bills & utilities', f'Month -3']
            cashflow.loc['Gambling spend', f'Month {i}'] = cashflow.loc['Gambling spend', f'Month -3']
            cashflow.loc['Payday spend', f'Month {i}'] = cashflow.loc['Payday spend', f'Month -3']
        elif i == -6 or i == -4 or i == -5:
            cashflow.loc['Broad income', f'Month {i}'] = (selected_row['total_income_broad_last_180_days'].iloc[0] - selected_row['total_income_broad_last_90_days'].iloc[0]) / 3
            cashflow.loc['Total benefits',f'Month {i}'] = (selected_row['total_benefits_last_180_days'].iloc[0] - selected_row['total_benefits_last_90_days'].iloc[0])/3
            cashflow.loc['Groceries spend',f'Month {i}'] = (selected_row['net_groceries_spend_last_180_days'].iloc[0] - selected_row['net_groceries_spend_last_90_days'].iloc[0])/3
            cashflow.loc['Mortgage & rent',f'Month {i}'] = (selected_row['net_mortgage_and_rent_spend_last_180_days'].iloc[0] - selected_row['net_mortgage_and_rent_spend_last_90_days'].iloc[0])/3
            cashflow.loc['Bills & utilities',f'Month {i}'] = (selected_row['net_bills_and_utilities_spend_last_180_days'].iloc[0] - selected_row['net_bills_and_utilities_spend_last_90_days'].iloc[0])/3
            cashflow.loc['Gambling spend',f'Month {i}'] = (selected_row['total_gambling_spend_last_180_days'].iloc[0] - selected_row['total_gambling_spend_last_90_days'].iloc[0])/3
            cashflow.loc['Payday spend',f'Month {i}'] = (selected_row['total_payday_spend_last_180_days'].iloc[0] - selected_row['total_payday_spend_last_90_days'].iloc[0])/3

    # These rows add the forecast figures              
    for i in fcast_periods:
        if i == 1:
            cashflow.loc['Broad income', f'Month {i}'] = cashflow.loc['Broad income', 'Month -1'] * (1 + inflation_df.loc['Wages', f'Month {i}'])
            cashflow.loc['Total benefits', f'Month {i}'] = cashflow.loc['Total benefits', 'Month -2'] * (1 + inflation_df.loc['Benefits', f'Month {i}'])
            cashflow.loc['Groceries spend', f'Month {i}'] = cashflow.loc['Groceries spend', 'Month -2'] * (1 + inflation_df.loc['CPI', f'Month {i}'])
            cashflow.loc['Bills & utilities', f'Month {i}'] = cashflow.loc['Bills & utilities', 'Month -2'] * (1 + inflation_df.loc['CPI', f'Month {i}'])
            cashflow.loc['Gambling spend', f'Month {i}'] = cashflow.loc['Gambling spend', 'Month -2'] * (1 + inflation_df.loc['CPI', f'Month {i}'])
            cashflow.loc['Payday spend', f'Month {i}'] = cashflow.loc['Payday spend', 'Month -2'] * (1 + inflation_df.loc['CPI', f'Month {i}']) 
            cashflow.loc['Mortgage & rent', f'Month {i}'] = cashflow.loc['Mortgage & rent', 'Month -2'] * (1 + inflation_df.loc['CPIH', f'Month {i}']) 
 
        if i in range(2, 25):
            cashflow.loc['Broad income', f'Month {i}'] = cashflow.loc['Broad income', f'Month {i - 1}'] * (1 + inflation_df.loc['Wages', f'Month {i}'])
            cashflow.loc['Total benefits', f'Month {i}'] = cashflow.loc['Total benefits', f'Month {i - 1}'] * (1 + inflation_df.loc['Benefits', f'Month {i}'])
            cashflow.loc['Groceries spend', f'Month {i}'] = cashflow.loc['Groceries spend', f'Month {i - 1}'] * (1 + inflation_df.loc['CPI', f'Month {i}'])
            cashflow.loc['Bills & utilities', f'Month {i}'] = cashflow.loc['Bills & utilities', f'Month {i - 1}'] * (1 + inflation_df.loc['CPI', f'Month {i}'])
            cashflow.loc['Gambling spend', f'Month {i}'] = cashflow.loc['Gambling spend', f'Month {i - 1}'] * (1 + inflation_df.loc['CPI', f'Month {i}'])
            cashflow.loc['Payday spend', f'Month {i}'] = cashflow.loc['Payday spend', f'Month {i - 1}'] * (1 + inflation_df.loc['CPI', f'Month {i}'])
            cashflow.loc['Mortgage & rent', f'Month {i}'] = cashflow.loc['Mortgage & rent', f'Month {i - 1}'] * (1 + inflation_df.loc['CPIH', f'Month {i}'])
            
    # These rows add the loan figures        
    selected_row2 = affordcheck_df[affordcheck_df['uuid'] == target_uuid]
    loan_pmt_value = selected_row2['PMT'].iloc[0]
    if pd.notnull(loan_pmt_value):  # Check if the value is not NaN
        cashflow.loc['LOAN PMT', :]  = - loan_pmt_value
            
    # Specify subtotals 
    cashflow.loc['Total income'] = cashflow.loc['Broad income'] + cashflow.loc['Total benefits']
    cashflow.loc['Total spending'] = (cashflow.loc['Groceries spend'] + cashflow.loc['Mortgage & rent'] 
                                  + cashflow.loc['Bills & utilities'] + cashflow.loc['Gambling spend'] + cashflow.loc['Payday spend'])
    
    # Specify b/f, EDI and c/f calculations    
    prev_bf = avg_balance 
    for i in range(-6, 25):
        if i == 0:
            continue  # Skip month 0
        if i in range(-5, 0):
            cashflow.loc['B/f', f'Month {i}'] = cashflow.loc['C/f', f'Month {i - 1}']
            prev_bf = cashflow.loc['B/f', f'Month {i}']
        elif i == 1:
            cashflow.loc['B/f', f'Month {i}'] = cashflow.loc['C/f', 'Month -1']
            prev_bf = cashflow.loc['B/f', f'Month {i}']
        elif i in range(2, 25):
            cashflow.loc['B/f', f'Month {i}'] = cashflow.loc['C/f', f'Month {i - 1}']
            prev_bf = cashflow.loc['B/f', f'Month {i}']
    
        EDI = prev_bf + cashflow.loc['Total income', f'Month {i}'] + cashflow.loc['Total spending', f'Month {i}']
        cashflow.loc['EDI', f'Month {i}'] = EDI
        CF = EDI + cashflow.loc['LOAN PMT', f'Month {i}']
        cashflow.loc['C/f', f'Month {i}'] = CF
    
    # Identify uuids where c/f is negative more than twice in time series
    count_negative_cf_columns = (cashflow.apply(lambda col: (col < 0)).sum())
    if (count_negative_cf_columns > 7).any():
        failed_affordability.append(target_uuid)
        
    #Identify gamblers
    if abs(cashflow.loc['Gambling spend'].any()) > cashflow.loc['Total income'].any():
        gamblers.append(target_uuid)
        
    # Identify drop in essential spending  
    if abs(cashflow.loc['Groceries spend', 'Month -4']) > abs(1.33 * -cashflow.loc['Groceries spend', 'Month -1']) or abs(cashflow.loc['Bills & utilities', 'Month -4']) > abs(1.54 * -cashflow.loc['Bills & utilities', 'Month -1']):
        drop_in_essential_spending.append(target_uuid)
    
    # Round values
    #cashflow = cashflow.round(0).astype(int)   #Rounding values in the dataframes is causing an exception error where NA values. Round in Excel instead.
    cashflow.iloc[0] = ''
    
    dataframe_outputs.append(cashflow)

'''for df in dataframe_outputs:
    print(df)
    print("\n" + "=" * 50 + "\n")  # Separating line for clarity '''

print('The following uuids failed the Open Banking affordability test:')
print (failed_affordability[:5])

print('///////////////////////////////////////////////////////////////')
print('The following uuids identified as excessive gamblers:')
print(gamblers[:5])

print('///////////////////////////////////////////////////////////////')
print('The following uuids identified as having a >25% drop in essential spending:')
print(drop_in_essential_spending[:5])

# If you want to view the full list of uuid's print the lines below
#print(drop_in_essential_spending)
#print(gamblers)
#print (failed_affordability)

The following uuids failed the Open Banking affordability test:
['2c274a67-6d4f-42d3-9016-059e2e73e690', '91f0323e-1017-46af-abf1-ea0bb359f2db', 'b1c21783-e3cd-4efd-9f41-e07afcf4ac94', 'dc07bf17-07d3-40cb-9409-d887aa9cc57f', 'b6c16d14-99b8-4d6c-bbeb-859a32d64a6b']
///////////////////////////////////////////////////////////////
The following uuids identified as excessive gamblers:
['48a29b8c-1d1a-41af-a45a-dc2d14023683', 'b6c16d14-99b8-4d6c-bbeb-859a32d64a6b', 'b1e99994-8a14-4ec3-b815-731fa69ce2a7', 'eaef1994-f0ca-4c1a-aa27-326e1864f6ee', '2b0aa0a7-c082-4f9d-87c8-bb612d3b49e1']
///////////////////////////////////////////////////////////////
The following uuids identified as having a >25% drop in essential spending:
['cb7b6ec0-a013-445e-a0dc-68114eca1251', 'bf2b9cb0-927a-4313-945f-72cadabe0e5c', '2c274a67-6d4f-42d3-9016-059e2e73e690', 'dc07bf17-07d3-40cb-9409-d887aa9cc57f', 'b6c16d14-99b8-4d6c-bbeb-859a32d64a6b']


In [12]:
# This cell copies and pastes the dataframes into an Excel workbook

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

wb = Workbook()
sheet = wb.active
for i, df in enumerate(dataframe_outputs):
    # Write the dataframe column headers to the worksheet
    columns = [None] + df.columns.tolist()  # Add None as the first element to offset by one column
    sheet.append(columns)
    for idx, row in df.iterrows():
        # Add index as the first column of each row
        row_data = [idx] + row.tolist()
        sheet.append(row_data)
    # Add an empty row after the dataframe
    sheet.append([])  # Empty row

# Set the width of column A
col_widths = {'A': 38}
for col, width in col_widths.items():
    sheet.column_dimensions[col].width = width
    
# Save the workbook with the specified filename
#import datetime
#current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')  # Format the datetime as a string
#filename = f'Cashflow output {current_datetime}.xlsx'
#wb.save(filename)
#print(f"Workbook saved as '{filename}'")

# Save the workbook with the specified filename
filename = 'Cashflow_Output_SpecificName.xlsx'  # Replace 'SpecificName' with your desired name
wb.save(filename)

print(f"Workbook saved as '{filename}'")

Workbook saved as 'Cashflow_Output_SpecificName.xlsx'


In [13]:
count_referrals = len(affordcheck_df)
count_valid_references = len(valid_references)
count_failed_check = len(failed_affordability)
count_gamblers = len(gamblers)
count_dropped_essential_spend = len(drop_in_essential_spending)

percentage_checked = (count_valid_references / count_referrals) * 100
pecentage_failed = (count_failed_check / count_valid_references) * 100
percentage_gamblers = (count_gamblers / count_valid_references) * 100
percentage_dropped_spend = (count_dropped_essential_spend / count_valid_references) * 100

print("Percentage checked for affordability: {:.0f}%".format(percentage_checked))
print("Percentage failing affordability check: {:.0f}%".format(pecentage_failed ))
print("Percentage excessive gambling: {:.0f}%".format(percentage_gamblers ))
print("Percentage dropped essential spending: {:.0f}%".format(percentage_dropped_spend))

Percentage checked for affordability: 40%
Percentage failing affordability check: 54%
Percentage excessive gambling: 36%
Percentage dropped essential spending: 40%
